### Emperical Mode Decomposition

In [1]:
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_pickle("data/data")

df_spo2 = df.iloc[:,4:2504]
df_spo2["label"] = df.iloc[:,-1]

df_spo2 = df_spo2[df_spo2.label != 0]
df_spo2.label.to_csv("data/spo2_label.csv")

In [ ]:
from PyEMD import EMD
import numpy as np

df_spo2_emd = df_spo2.drop(["label"],axis=1)
emd = EMD()
emd_result = []
for i in tqdm(range(df_spo2_emd.shape[0])):
    emd_result.append(emd(df_spo2_emd.iloc[i,:].values))

In [ ]:
X_train = []
for i in tqdm(range(len(emd_result))):
    X_train.append(np.array([df_spo2_emd.iloc[i,:].tolist(),emd_result[i][0],emd_result[i][1],emd_result[i][2]]))

In [ ]:
import torch
X = torch.tensor(np.array(X_train))

torch.save(X,"data/EMD_spo_train_3.pt")

### Inceptiontime Model

In [ ]:
### Need to start a new nb
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [ ]:
import pdb
import numpy as np
import pandas as pd
#from joblib import Parallel, delayed
import multiprocessing
from collections import Counter
import pickle
import tables
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from numpy import genfromtxt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
import torch
from tqdm import tqdm

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
np.set_printoptions(threshold=np.inf)
from tsai.all import *

In [ ]:
X = torch.load("data/EMD_spo_train_3.pt",map_location = torch.device('cpu')).type(torch.FloatTensor)
spo2_label = pd.read_csv("data/spo2_label.csv")

splits = get_splits(y, valid_size=.3, show_plot=False)

In [ ]:
check_data(X, y, splits)

In [ ]:
tfms = [None, TSRegression()]
batch_tfms = TSStandardize(by_sample=True)

In [ ]:
dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms)

In [ ]:
### pre-training
learn = ts_learner(dls, InceptionTimePlus, cbs=[ShowGraph(), MVP(target_dir='./data/MVP', fname="ITP_200visits_all")])
learn.fit_one_cycle(100, 1e-2)

In [ ]:
n_epochs = 85
learn = ts_learner(dls, InceptionTimePlus, pretrained=True, weights_path=f'data/MVP/ITP_200visits_all.pth', metrics=mae, cbs=CutMix1D())
learn.fit_one_cycle(n_epochs, 6e-3)

In [ ]:
PATH = Path('./models/SPO2_pretraining_oridata1.pkl')
PATH.parent.mkdir(parents=True, exist_ok=True)
learn.export(PATH)